# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import geoapify_key

In [29]:
#corrected-data_path = 'weatherAPI_data.csv'
corrected_data_path = "../output_data/cities.csv"

corrected_data_df = pd.read_csv(corrected_data_path)
corrected_data_df.head(20)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,2075265,Busselton,0,AU,1680458211,92,-33.6500,115.3333,59.36,9.15
1,2126123,Chokurdakh,47,RU,1680458213,98,70.6333,147.9167,-17.93,9.51
2,5848280,Kapaa,75,US,1680458214,93,22.0752,-159.3190,71.92,5.75
3,4155996,Fort Myers Beach,0,US,1680458215,70,26.4520,-81.9481,89.06,9.22
4,1282256,Hithadhoo,58,MV,1680458216,72,-0.6000,73.0833,82.72,7.70
5,5866583,Saint Paul Harbor,0,US,1680458033,59,57.7900,-152.4072,31.51,11.50
6,5122534,Jamestown,0,US,1680458218,37,42.0970,-79.2353,39.52,9.22
7,4030556,Rikitea,45,PF,1680458220,80,-23.1203,-134.9692,80.53,14.63
8,3430863,Mar del Plata,0,AR,1680458221,45,-38.0023,-57.5575,77.95,6.91
9,359792,Aswān,0,EG,1680458222,13,24.0934,32.9070,81.70,6.91


In [24]:
gmaps.configure(api_key=geoapify_key)
locations = corrected_data_df[["Lat","Lng"]]
humidity = corrected_data_df["Humidity"]

In [23]:
fig = gmaps.figure(center=(50.0,-5.0), zoom_level=1)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [22]:
#make a new df with cities with 0 cloudiness, wind speed <10 and humidity<20
new_df = corrected_data_df[(corrected_data_df["Cloudiness"] == 0) & (corrected_data_df["Wind Speed"] <= 10) & (corrected_data_df["Humidity"] <= 20)].dropna()
new_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
96,96,zhangye,38.9342,100.4517,11.11,16,0,1.67,CN,1666108292
436,436,lakki marwat,32.6079,70.9114,27.89,20,0,1.53,PK,1666108492
476,476,chifeng,42.2683,118.9636,5.09,20,0,2.81,CN,1666108512
496,496,tikrit,34.6071,43.6782,29.02,14,0,1.41,IQ,1666108524
542,542,bilma,18.6853,12.9164,31.90,18,0,4.17,NE,1666108548


### : Create a new DataFrame called `hotel_df`.

In [19]:
#adda column name Hotel_Name
hotel_df = new_df.loc[:,["City_ID", "City", "Country", "Lat", "Lng"]]
hotel_df
hotel_df["Hotel_Name"]=""
hotel_df

,City_ID,City,Country,Lat,Lng,Hotel_Name
96,96,zhangye,CN,38.9342,100.4517,
436,436,lakki marwat,PK,32.6079,70.9114,
476,476,chifeng,CN,42.2683,118.9636,
496,496,tikrit,IQ,34.6071,43.6782,
542,542,bilma,NE,18.6853,12.9164,


In [34]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000 , 
    "keyword": "hotel",
    "types": "lodging",
    "key": geoapify_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    
    # get hotel location
    lat = row['Lat']
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params['location'] = f"{lat},{lng}"

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    #print(response)
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel_Name'] = results[0]['name']
       # print(results[0]['name'])
        #types_df.loc[index, 'address'] = results[0]['vicinity']
        #types_df.loc[index, 'price_level'] = results[0]['price_level']
        #types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------


In [36]:
hotel_df

,City_ID,City,Country,Lat,Lng,Hotel_Name
96,96,zhangye,CN,38.9342,100.4517,
436,436,lakki marwat,PK,32.6079,70.9114,
476,476,chifeng,CN,42.2683,118.9636,
496,496,tikrit,IQ,34.6071,43.6782,
542,542,bilma,NE,18.6853,12.9164,


In [37]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# Add marker layer ontop of heat map
markerlayer=gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markerlayer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))